<a href="https://colab.research.google.com/github/j143/notebooks/blob/master/Recommendation_with_Pyspark%2BSystemDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright &copy; 2020 The Apache Software Foundation.

In [1]:
# @title Apache Version 2.0 (The "License");
#-------------------------------------------------------------
#
# Licensed to the Apache Software Foundation (ASF) under one
# or more contributor license agreements.  See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership.  The ASF licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License.  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express or implied.  See the License for the
# specific language governing permissions and limitations
# under the License.
#
#-------------------------------------------------------------

### Developer notebook for Apache SystemDS

Run this notebook online at [Google Colab ↗](https://colab.research.google.com/github/apache/systemds/blob/master/notebooks/systemds_dev.ipynb).




This Jupyter/Colab-based tutorial will interactively walk through development setup and running SystemDS in both the

A. standalone mode \
B. with Apache Spark.

Flow of the notebook:
1. Download and Install the dependencies
2. Go to section **A** or **B**

#### Download and Install the dependencies

1. **Runtime:** Java (OpenJDK 8 is preferred)
2. **Build:** Apache Maven
3. **Backend:** Apache Spark (optional)

##### Setup

A custom function to run OS commands.

In [2]:
# Run and print a shell command.
def run(command):
  print('>> {}'.format(command))
  !{command}
  print('')

##### Install Java
Let us install OpenJDK 8. More about [OpenJDK ↗](https://openjdk.java.net/install/).

In [3]:
!apt-get update

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-s

In [4]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# run the below command to replace the existing installation
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

!java -version

update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
openjdk version "1.8.0_265"
OpenJDK Runtime Environment (build 1.8.0_265-8u265-b01-0ubuntu2~18.04-b01)
OpenJDK 64-Bit Server VM (build 25.265-b01, mixed mode)


##### Install Apache Spark (Optional, if you want to work with spark backend)


NOTE: If spark is not downloaded. Let us make sure the version we are trying to download is officially supported at
https://spark.apache.org/downloads.html

In [5]:
# Spark and Hadoop version
spark_version = 'spark-2.4.6'
hadoop_version = 'hadoop2.7'
spark_path = f"/opt/{spark_version}-bin-{hadoop_version}"
if not os.path.exists(spark_path):
  run(f"wget -q -nc -O apache-spark.tgz https://downloads.apache.org/spark/{spark_version}/{spark_version}-bin-{hadoop_version}.tgz")
  run('tar zxf apache-spark.tgz -C /opt')
  run('rm -f apache-spark.tgz')

os.environ["SPARK_HOME"] = spark_path
os.environ["PATH"] += ":$SPARK_HOME/bin"


>> wget -q -nc -O apache-spark.tgz https://downloads.apache.org/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz

>> tar zxf apache-spark.tgz -C /opt

>> rm -f apache-spark.tgz



In [7]:
!pip install -q findspark

In [8]:
import findspark
findspark.init()

In [9]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("systemds").setMaster("local[*]")
sc = SparkContext(conf=conf)

In [17]:
from pyspark.sql import SparkSession

spark = SparkSession(sc)

#### Get Apache SystemDS

Apache SystemDS development happens on GitHub at [apache/systemds ↗](https://github.com/apache/systemds)

In [10]:
!git clone https://github.com/apache/systemds systemds --depth=1
%cd systemds

Cloning into 'systemds'...
remote: Enumerating objects: 7642, done.
remote: Counting objects: 100% (7642/7642), done.
remote: Compressing objects: 100% (4588/4588), done.
remote: Total 7642 (delta 5595), reused 3643 (delta 2967), pack-reused 0
Receiving objects: 100% (7642/7642), 14.79 MiB | 11.47 MiB/s, done.
Resolving deltas: 100% (5595/5595), done.
/content/systemds


In [11]:
%%sh

# Download dataset
curl -O http://snap.stanford.edu/data/amazon0601.txt.gz
gunzip amazon0601.txt.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11.2M  100 11.2M    0     0  2825k      0  0:00:04  0:00:04 --:--:-- 2825k


In [12]:
!sed -n 1,10p amazon0601.txt

# Directed graph (each unordered pair of nodes is saved once): Amazon0601.txt 
# Amazon product co-purchaisng network from June 01 2003
# Nodes: 403394 Edges: 3387388
# FromNodeId	ToNodeId
0	1
0	2
0	3
0	4
0	5
0	6


In [13]:
import pyspark.sql.functions as F


In [19]:
!ls

amazon0601.txt	conf		 dev	 docs	  notebooks  pom.xml	scripts
bin		CONTRIBUTING.md  docker  LICENSE  NOTICE     README.md	src


In [21]:
dataPath = "/content/amazon0601.txt"

X_train = (sc.textFile(dataPath)
    .filter(lambda l: not l.startswith("#"))
    .map(lambda l: l.split("\t"))
    .map(lambda prods: (int(prods[0]), int(prods[1]), 1.0))
    .toDF(("prod_i", "prod_j", "x_ij"))
    .filter("prod_i < 500 AND prod_j < 500") # Filter for memory constraints
    .cache())

In [22]:
X_train

DataFrame[prod_i: bigint, prod_j: bigint, x_ij: double]

In [23]:
# Maximum product id, is 499
max_prod_i = X_train.select(F.max("prod_i")).first()[0]

In [25]:
max_prod_j = X_train.select(F.max("prod_j")).first()[0]

In [27]:
numProducts = max(max_prod_i, max_prod_j) + 1 # since 0-based indexing

In [28]:
print("Total number of products: {}".format(numProducts))

Total number of products: 500
